In [1]:
import sys
import sqlite3
import threading
import time
import requests
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QTableView, QVBoxLayout, QPushButton,
    QLineEdit, QWidget, QHBoxLayout, QFormLayout, QDialog, QLabel, QMessageBox, QProgressBar
)
from PyQt5.QtCore import QAbstractTableModel, Qt, QTimer, pyqtSignal

In [2]:
class PostsTableModel(QAbstractTableModel):
    def __init__(self, data):
        super().__init__()
        self._data = data

    def data(self, index, role):
        if role == Qt.DisplayRole:
            return self._data[index.row()][index.column()]

    def rowCount(self, index):
        return len(self._data)

    def columnCount(self, index):
        return len(self._data[0])

    def headerData(self, section, orientation, role):
        headers = ["ID", "User ID", "Title", "Body"]
        if role == Qt.DisplayRole:
            if orientation == Qt.Horizontal:
                return headers[section]


class MainWindow(QMainWindow):
    progress_update = pyqtSignal(str)  # Сигнал для обновления текста прогресса
    progress_value = pyqtSignal(int)   # Сигнал для обновления прогресса в виде числа

    def __init__(self):
        super().__init__()

        self.conn = sqlite3.connect('labDataBase.db')
        self.cursor = self.conn.cursor()

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)

        self.table_view = QTableView()
        self.search_bar = QLineEdit()
        self.search_bar.setPlaceholderText("Поиск по заголовку...")
        self.refresh_button = QPushButton("Обновить")
        self.add_button = QPushButton("Добавить")
        self.delete_button = QPushButton("Удалить")
        self.load_button = QPushButton("Загрузить данные с сервера")
        self.progress_bar = QProgressBar()

        layout = QVBoxLayout()
        button_layout = QHBoxLayout()

        button_layout.addWidget(self.refresh_button)
        button_layout.addWidget(self.add_button)
        button_layout.addWidget(self.delete_button)
        button_layout.addWidget(self.load_button)

        layout.addWidget(self.search_bar)
        layout.addWidget(self.table_view)
        layout.addLayout(button_layout)
        layout.addWidget(self.progress_bar)

        self.central_widget.setLayout(layout)

        self.refresh_button.clicked.connect(self.refresh_data)
        self.add_button.clicked.connect(self.show_add_dialog)
        self.delete_button.clicked.connect(self.delete_record)
        self.load_button.clicked.connect(self.load_data_from_server)
        self.search_bar.textChanged.connect(self.search_title)

        self.progress_update.connect(self.update_progress_text)  # Подключаем сигнал
        self.progress_value.connect(self.update_progress_bar)  # Подключаем сигнал для обновления прогресс-бара

        self.timer = QTimer(self)
        self.timer.timeout.connect(self.load_data_from_server)
        self.timer.setInterval(10000)  # Таймер будет срабатывать каждые 10 секунд
        self.timer.start()

        self.loading_data = False  # Флаг загрузки данных

        self.refresh_data()

    def refresh_data(self):
        query = "SELECT * FROM posts"
        self.cursor.execute(query)
        records = self.cursor.fetchall()
        self.update_table(records)

    def update_table(self, records):
        self.model = PostsTableModel(records)
        self.table_view.setModel(self.model)

    def search_title(self):
        search_text = self.search_bar.text()
        query = "SELECT * FROM posts WHERE title LIKE ?"
        self.cursor.execute(query, ('%' + search_text + '%',))
        records = self.cursor.fetchall()
        self.update_table(records)

    def show_add_dialog(self):
        dialog = AddPostDialog(self)
        dialog.exec_()

    def add_post(self, user_id, title, body):
        # Проверка на дублирование записи
        query = "SELECT COUNT(*) FROM posts WHERE user_id = ? AND title = ? AND body = ?"
        self.cursor.execute(query, (user_id, title, body))
        if self.cursor.fetchone()[0] == 0:  # Если таких записей нет
            query = "INSERT INTO posts (user_id, title, body) VALUES (?, ?, ?)"
            self.cursor.execute(query, (user_id, title, body))
            self.conn.commit()
            self.refresh_data()

    def delete_record(self):
        selected = self.table_view.selectionModel().selectedRows()
        if selected:
            reply = QMessageBox.question(self, "Подтверждение", "Удалить выбранную запись?",
                                         QMessageBox.Yes | QMessageBox.No, QMessageBox.No)
            if reply == QMessageBox.Yes:
                for index in selected:
                    post_id = self.model.data(index, Qt.DisplayRole)
                    query = "DELETE FROM posts WHERE id = ?"
                    self.cursor.execute(query, (post_id,))
                self.conn.commit()
                self.refresh_data()

    def load_data_from_server(self):
        if self.loading_data:  # Если данные уже загружаются, не начинаем новый процесс загрузки
            return

        self.loading_data = True  # Устанавливаем флаг загрузки
        self.progress_bar.setValue(0)
        self.progress_update.emit("Загружаем данные с сервера...")

        thread = threading.Thread(target=self.fetch_and_save_data)
        thread.start()

    def fetch_and_save_data(self):
        url = "https://jsonplaceholder.typicode.com/posts"
        response = requests.get(url)
        posts = response.json()

        conn_thread = sqlite3.connect('labDataBase.db')
        cursor_thread = conn_thread.cursor()

        self.progress_update.emit("Сохранение данных в базу данных...")
        for i, post in enumerate(posts):
            self.save_post_to_db(cursor_thread, post)
            self.progress_value.emit(int((i + 1) / len(posts) * 100))
            time.sleep(0.1)

        conn_thread.commit()

        query = "SELECT * FROM posts"
        cursor_thread.execute(query)
        records = cursor_thread.fetchall()
        self.update_table(records)

        conn_thread.close()

        self.progress_update.emit("Данные успешно загружены и сохранены.")
        self.loading_data = False  # Сбрасываем флаг загрузки

    def save_post_to_db(self, cursor, post):
        """Сохранение поста в базу данных, проверка на наличие дубликатов."""
        query = "SELECT COUNT(*) FROM posts WHERE user_id = ? AND title = ? AND body = ?"
        cursor.execute(query, (post['userId'], post['title'], post['body']))
        if cursor.fetchone()[0] == 0:  # Если запись не существует
            query = "INSERT INTO posts (user_id, title, body) VALUES (?, ?, ?)"
            cursor.execute(query, (post['userId'], post['title'], post['body']))

    def update_progress_text(self, text):
        """Обновление текста прогресса."""
        self.statusBar().showMessage(text)

    def update_progress_bar(self, value):
        """Обновление значения прогресс-бара."""
        self.progress_bar.setValue(value)


class AddPostDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Добавить запись")

        self.user_id_input = QLineEdit()
        self.title_input = QLineEdit()
        self.body_input = QLineEdit()

        self.add_button = QPushButton("Добавить")
        self.add_button.clicked.connect(self.add_post)

        layout = QFormLayout()
        layout.addRow("User ID:", self.user_id_input)
        layout.addRow("Title:", self.title_input)
        layout.addRow("Body:", self.body_input)
        layout.addRow(self.add_button)

        self.setLayout(layout)

    def add_post(self):
        user_id = self.user_id_input.text()
        title = self.title_input.text()
        body = self.body_input.text()

        if user_id and title and body:
            self.parent().add_post(user_id, title, body)
            self.accept()
        else:
            QMessageBox.warning(self, "Ошибка", "Заполните все поля")


In [3]:
if __name__ == "__main__":
    app = QApplication(sys.argv)

    window = MainWindow()
    window.setWindowTitle("Lab 5")
    window.resize(800, 600)
    window.show()

    sys.exit(app.exec_())

SystemExit: 0

C:\Users\Sergey\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
